In [3]:
!nvidia-smi

Sat May 11 10:48:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Install required libraries

In [4]:
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3
Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Successfully uninstalled accelerate-0.30.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.0 MB/s eta 0:00:0000:0100:01
Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalli

# Importing reuired libraries

In [2]:
%env WANDB_DISABLED=True

from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from tqdm import tqdm
import torch
import wandb
import huggingface_hub
import warnings
warnings.filterwarnings("ignore")
from kaggle_secrets import UserSecretsClient
from IPython.display import FileLink
nltk.download("punkt")

env: WANDB_DISABLED=True
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [22]:
# Fetch Secrets
user_secrets=UserSecretsClient()

# Fetch secret keys
my_secret_hf_api_key_read=user_secrets.get_secret("hf_read")
my_secret_hf_api_key_write=user_secrets.get_secret("hf_write")
my_secret_wandb_api_key=user_secrets.get_secret("wandb_api_key")

# Use the secret keys to login
wandb.login(key=my_secret_wandb_api_key)
huggingface_hub.login(token=my_secret_hf_api_key_read, add_to_git_credential=False)
huggingface_hub.login(token=my_secret_hf_api_key_write, add_to_git_credential=False)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Data Loading

## Loading model and tokenizer

In [4]:
model_ckpt = "google/pegasus-cnn_dailymail"
# loading in the tokenizers
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# initializing the model
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

## Loading the Dataset (Samsum)

In [5]:
dataset_samsum = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [6]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [7]:
dataset_samsum['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [8]:
dataset_samsum['train']['id'][0]

'13818513'

In [9]:
print(dataset_samsum["train"]["dialogue"][1])

Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great


In [10]:
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

In [11]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]
print([name for name in dataset_samsum])

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["train"][1]["dialogue"])
print("\nSummary:")
print(dataset_samsum["train"][1]["summary"])

['train', 'test', 'validation']
Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

Summary:
Olivia and Olivier are voting for liberals in this election. 


In [12]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
#     target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [13]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [15]:
dataset_samsum_pt["train"]["attention_mask"][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [16]:
dataset_samsum_pt["train"]["summary"][1]

'Olivia and Olivier are voting for liberals in this election. '

# Model Building

### Data Collator Utility : 
The DataCollatorForSeq2Seq class is a utility provided by the Hugging Face Transformers library for processing data during training of sequence-to-sequence models. It's particularly useful when you're working with sequence-to-sequence (seq2seq) models like Pegasus for tasks such as text summarization or translation.

Here's what this function does:

    DataCollatorForSeq2Seq: This class is responsible for collating and preprocessing batches of data for seq2seq models. It helps in handling inputs and outputs of varying lengths and converting them into proper tensors that can be fed into the model during training.

    tokenizer: This argument is an instance of a tokenizer class, such as PegasusTokenizer, provided by the Transformers library. The tokenizer is used to preprocess the input text data and convert it into tokens that the model can understand.

    model=model_pegasus: This specifies the seq2seq model that will be used for training. In this case, model_pegasus is presumably an instance of a Pegasus model provided by the Transformers library. The DataCollatorForSeq2Seq class can use this information to perform any necessary preprocessing specific to the model architecture.

When you create an instance of DataCollatorForSeq2Seq with these arguments, you get an object that can be used to collate and preprocess batches of input-output pairs for training your seq2seq model. This is crucial for handling tasks like padding sequences to the same length, converting tokens to IDs, and preparing attention masks.

## Setting Training Args


In [25]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

trainer_args = TrainingArguments(output_dir='pegasus-samsum', 
                                 num_train_epochs=1, 
                                 warmup_steps=500,
                                 per_device_train_batch_size=1, 
                                 per_device_eval_batch_size=1,
                                 weight_decay=0.01, 
                                 logging_steps=10,
                                 eval_steps=500, save_steps=1e6,
                                 gradient_accumulation_steps=16,
                                 push_to_hub = True,
                                 hub_private_repo = True,
                                 hub_model_id = 'Text_Summarizer',
                                 learning_rate=1e-5,
                                 warmup_ratio=0.1,   
                                 evaluation_strategy='epoch',
                                 do_eval=True,
                                 do_train=True,
                                 do_predict=True,
                                )

trainer = Trainer(model=model_pegasus, 
                  args=trainer_args,
                  tokenizer=tokenizer, 
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"]
                 )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
# training the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,3.393100,2.591580


TrainOutput(global_step=25, training_loss=3.1880844497680663, metrics={'train_runtime': 377.9768, 'train_samples_per_second': 2.167, 'train_steps_per_second': 0.066, 'total_flos': 423944250507264.0, 'train_loss': 3.1880844497680663, 'epoch': 0.98})

In [31]:
# saving the trained model
trainer.save_model('pegasus_text_summarizer.bin')

In [32]:
# !zip -r file.zip /kaggle/working
#Link to save the model
FileLink(r'file.zip')

/kaggle/working/file.zip

In [33]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [40]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [41]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

In [39]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model=model_pegasus,tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
